# Phase 2: Vessel Segmentation (Simplified)

**⚠️ THIS PHASE IS OPTIONAL - You can skip directly to Phase 3!**

This notebook uses the OCTA-autosegmentation model to segment vessels from the MIP en-face projection.

## When to run this notebook:
- ✅ If you need vessel density analysis
- ✅ If you need vessel morphology metrics
- ✅ If you're studying pathology (vessel dropout, etc.)

## When to skip this notebook:
- ❌ If you only need volume registration (Phase 3 uses MIP directly from Phase 1)

**Note:** We no longer need bifurcation detection! Phase 3 uses phase correlation directly on MIP images.

## Workflow:
1. Load MIP from Phase 1
2. Prepare image for OCTA-autosegmentation (resize to 304×304)
3. Run pretrained model
4. Extract vessel mask and skeleton
5. Save results (mask + skeleton only, no bifurcations needed!)

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import cv2
import subprocess
from skimage.morphology import skeletonize

# Setup paths
data_dir = Path('data')
octa_seg_repo = Path('/home/aristarx/Diploma/OCTA-autosegmentation')

print("✓ Imports complete")

## Helper Functions

In [ ]:
def prepare_image_for_octa_seg(enface_mip, target_size=(304, 304)):
    """
    Prepare MIP en-face image for OCTA-autosegmentation model.
    
    Args:
        enface_mip: MIP en-face projection (W, Z)
        target_size: Target size for model input
    
    Returns:
        img_prepared: Prepared image for model (H, W)
    """
    # Normalize to 0-255
    img_norm = ((enface_mip - enface_mip.min()) / 
                (enface_mip.max() - enface_mip.min()) * 255).astype(np.uint8)
    
    # Transpose to (Z, W) for standard image orientation
    img_norm = img_norm.T
    
    # Resize to model input size
    img_resized = cv2.resize(img_norm, target_size, interpolation=cv2.INTER_LINEAR)
    
    return img_resized

def run_octa_segmentation(input_image_path, output_dir, octa_seg_repo):
    """
    Run OCTA-autosegmentation on an image.
    
    Args:
        input_image_path: Path to input PNG
        output_dir: Directory for output
        octa_seg_repo: Path to OCTA-autosegmentation repository
    
    Returns:
        success: Boolean indicating success
    """
    config_file = octa_seg_repo / 'docker/trained_models/ves_seg-S-GAN/config.yml'
    abs_input_dir = input_image_path.parent
    abs_output_dir = Path.cwd() / output_dir
    
    cmd = [
        sys.executable,
        str(octa_seg_repo / 'test.py'),
        '--config_file', str(config_file),
        '--Test.data.image.files', f'{abs_input_dir}/**/*.png',
        '--Test.save_dir', str(abs_output_dir),
        '--epoch', '30',
        '--General.device', 'cpu'
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True, cwd=str(octa_seg_repo))
    return result.returncode == 0

print("✓ Helper functions defined")

## Process Volume 0

In [ ]:
# Load MIP from Phase 1
print("Loading Volume 0 MIP...")
enface_mip_v0 = np.load(data_dir / 'enface_mip_volume0.npy')
print(f"  Shape: {enface_mip_v0.shape}")

# Prepare for segmentation
print("\nPreparing image...")
img_prepared_v0 = prepare_image_for_octa_seg(enface_mip_v0)
print(f"  Prepared shape: {img_prepared_v0.shape}")

# Save input
input_dir_v0 = data_dir / 'octa_seg_input'
output_dir_v0 = data_dir / 'octa_seg_output'
input_dir_v0.mkdir(exist_ok=True, parents=True)
output_dir_v0.mkdir(exist_ok=True, parents=True)

input_path_v0 = input_dir_v0 / 'volume0_mip.png'
cv2.imwrite(str(input_path_v0), img_prepared_v0)
print(f"  Saved to: {input_path_v0}")

# Run segmentation
print("\nRunning OCTA-autosegmentation...")
success = run_octa_segmentation(input_path_v0, output_dir_v0, octa_seg_repo)

if success:
    print("✓ Segmentation complete!")
else:
    print("✗ Segmentation failed!")

In [ ]:
# Load segmentation results
seg_files_v0 = list(output_dir_v0.glob('**/*.png'))

if seg_files_v0:
    print(f"Found {len(seg_files_v0)} segmentation result(s)")
    
    # Load and convert to binary
    vessel_seg_v0 = cv2.imread(str(seg_files_v0[0]), cv2.IMREAD_GRAYSCALE)
    vessel_mask_v0 = vessel_seg_v0 > 127
    
    # Extract skeleton
    vessel_skeleton_v0 = skeletonize(vessel_mask_v0)
    
    print(f"\n✓ Processed Volume 0:")
    print(f"  Vessel mask: {vessel_mask_v0.shape}")
    print(f"  Vessel coverage: {100*vessel_mask_v0.sum()/vessel_mask_v0.size:.2f}%")
    print(f"  Skeleton length: {vessel_skeleton_v0.sum()} pixels")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(img_prepared_v0, cmap='gray')
    axes[0].set_title('Input MIP')
    axes[0].axis('off')
    
    axes[1].imshow(vessel_mask_v0, cmap='gray')
    axes[1].set_title('Vessel Mask')
    axes[1].axis('off')
    
    axes[2].imshow(img_prepared_v0, cmap='gray', alpha=0.7)
    axes[2].imshow(vessel_mask_v0, cmap='Reds', alpha=0.3)
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.suptitle('Volume 0 Vessel Segmentation', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠ No segmentation results found!")

## Process Volume 1

In [ ]:
# Load MIP from Phase 1
print("Loading Volume 1 MIP...")
enface_mip_v1 = np.load(data_dir / 'enface_mip_volume1.npy')
print(f"  Shape: {enface_mip_v1.shape}")

# Prepare for segmentation
print("\nPreparing image...")
img_prepared_v1 = prepare_image_for_octa_seg(enface_mip_v1)
print(f"  Prepared shape: {img_prepared_v1.shape}")

# Save input
input_dir_v1 = data_dir / 'octa_seg_input_v1'
output_dir_v1 = data_dir / 'octa_seg_output_v1'
input_dir_v1.mkdir(exist_ok=True, parents=True)
output_dir_v1.mkdir(exist_ok=True, parents=True)

input_path_v1 = input_dir_v1 / 'volume1_mip.png'
cv2.imwrite(str(input_path_v1), img_prepared_v1)
print(f"  Saved to: {input_path_v1}")

# Run segmentation
print("\nRunning OCTA-autosegmentation...")
success = run_octa_segmentation(input_path_v1, output_dir_v1, octa_seg_repo)

if success:
    print("✓ Segmentation complete!")
else:
    print("✗ Segmentation failed!")

In [ ]:
# Load segmentation results
seg_files_v1 = list(output_dir_v1.glob('**/*.png'))

if seg_files_v1:
    print(f"Found {len(seg_files_v1)} segmentation result(s)")
    
    # Load and convert to binary
    vessel_seg_v1 = cv2.imread(str(seg_files_v1[0]), cv2.IMREAD_GRAYSCALE)
    vessel_mask_v1 = vessel_seg_v1 > 127
    
    # Extract skeleton
    vessel_skeleton_v1 = skeletonize(vessel_mask_v1)
    
    print(f"\n✓ Processed Volume 1:")
    print(f"  Vessel mask: {vessel_mask_v1.shape}")
    print(f"  Vessel coverage: {100*vessel_mask_v1.sum()/vessel_mask_v1.size:.2f}%")
    print(f"  Skeleton length: {vessel_skeleton_v1.sum()} pixels")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    axes[0].imshow(img_prepared_v1, cmap='gray')
    axes[0].set_title('Input MIP')
    axes[0].axis('off')
    
    axes[1].imshow(vessel_mask_v1, cmap='gray')
    axes[1].set_title('Vessel Mask')
    axes[1].axis('off')
    
    axes[2].imshow(img_prepared_v1, cmap='gray', alpha=0.7)
    axes[2].imshow(vessel_mask_v1, cmap='Reds', alpha=0.3)
    axes[2].set_title('Overlay')
    axes[2].axis('off')
    
    plt.suptitle('Volume 1 Vessel Segmentation', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠ No segmentation results found!")

## Side-by-Side Comparison

In [ ]:
if seg_files_v0 and seg_files_v1:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Volume 0 row
    axes[0, 0].imshow(img_prepared_v0, cmap='gray')
    axes[0, 0].set_title('Volume 0: MIP')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(vessel_mask_v0, cmap='gray')
    axes[0, 1].set_title(f'Volume 0: Vessel Mask\n{100*vessel_mask_v0.sum()/vessel_mask_v0.size:.1f}% coverage')
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(vessel_skeleton_v0, cmap='gray')
    axes[0, 2].set_title(f'Volume 0: Skeleton\n{vessel_skeleton_v0.sum()} pixels')
    axes[0, 2].axis('off')
    
    # Volume 1 row
    axes[1, 0].imshow(img_prepared_v1, cmap='gray')
    axes[1, 0].set_title('Volume 1: MIP')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(vessel_mask_v1, cmap='gray')
    axes[1, 1].set_title(f'Volume 1: Vessel Mask\n{100*vessel_mask_v1.sum()/vessel_mask_v1.size:.1f}% coverage')
    axes[1, 1].axis('off')
    
    axes[1, 2].imshow(vessel_skeleton_v1, cmap='gray')
    axes[1, 2].set_title(f'Volume 1: Skeleton\n{vessel_skeleton_v1.sum()} pixels')
    axes[1, 2].axis('off')
    
    plt.suptitle('Vessel Segmentation Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## Save Results

In [ ]:
if seg_files_v0 and seg_files_v1:
    # Save Volume 0
    np.save(data_dir / 'vessel_mask_volume0.npy', vessel_mask_v0)
    np.save(data_dir / 'vessel_skeleton_volume0.npy', vessel_skeleton_v0)
    
    # Save Volume 1
    np.save(data_dir / 'vessel_mask_volume1.npy', vessel_mask_v1)
    np.save(data_dir / 'vessel_skeleton_volume1.npy', vessel_skeleton_v1)
    
    print("✓ Saved vessel segmentation results:")
    print(f"  - vessel_mask_volume0.npy: {vessel_mask_v0.shape}")
    print(f"  - vessel_skeleton_volume0.npy: {vessel_skeleton_v0.shape}")
    print(f"  - vessel_mask_volume1.npy: {vessel_mask_v1.shape}")
    print(f"  - vessel_skeleton_volume1.npy: {vessel_skeleton_v1.shape}")
    
    print("\n📊 Summary:")
    print(f"  Volume 0:")
    print(f"    - Vessel coverage: {100*vessel_mask_v0.sum()/vessel_mask_v0.size:.2f}%")
    print(f"    - Skeleton length: {vessel_skeleton_v0.sum()} pixels")
    print(f"  Volume 1:")
    print(f"    - Vessel coverage: {100*vessel_mask_v1.sum()/vessel_mask_v1.size:.2f}%")
    print(f"    - Skeleton length: {vessel_skeleton_v1.sum()} pixels")
    
    print("\n✓ Phase 2 vessel segmentation complete!")
    print("\n💡 Note: No bifurcation detection needed!")
    print("   Phase 3 uses phase correlation directly on MIP images for registration.")
    print("\nNext step:")
    print("  Phase 3: XY registration using phase correlation (03_xy_registration_phase_correlation.ipynb)")
else:
    print("⚠ Cannot save - segmentation failed for one or both volumes")